In [2]:
!pip3 install tensorflow

In [3]:
!pip3 install pandas_datareader


In [4]:
!pip3 install yfinance

In [5]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import yfinance as yf


from tqdm import tqdm_notebook, tqdm
from collections import deque

In [6]:
tf.__version__

'2.15.0'

In [7]:
class AI_Trader():
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen =2000)
    self.model_name = model_name

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.model_builder()

  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
    model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001))
    return model

  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state)
    return np.argmax(actions[0])

  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])

    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state, target, epochs=1, verbose=0)

    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [8]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [9]:
# sigmoid(0.5)

In [10]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [11]:
# stocks_price_format(100)

In [12]:
dataset = yf.download("AAPL", start="2013-01-01", end="2023-12-31")

[*********************100%%**********************]  1 of 1 completed


In [13]:
# dataset.head()

In [14]:
# str(dataset.index[0]).split()[0]

In [15]:
# dataset.index[-1]

In [16]:
# dataset['Close']

In [17]:
def dataset_loader(stock_name, start=None, end=None):
  dataset = yf.download(stock_name, start=start, end=end)
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close


In [18]:
# 0 - 5 + 1

In [19]:
# 20 - 5 + 1

In [20]:
# dataset[16:21]

In [21]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))

  return np.array([state]), windowed_data

In [22]:
stock_name = "AAPL"
start_date = "2013-01-01"
end_date = "2023-12-31"
data = dataset_loader(stock_name, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [23]:
# s, w = state_creator(data, 0, 5)

In [24]:
# s

In [25]:
# w

In [26]:
window_size = 10
episodes = 10
batch_size = 32
data_samples = len(data) -1

In [27]:
# data_samples

In [28]:
trader = AI_Trader(window_size)

In [29]:
# trader.model.summary()

In [31]:
for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0

    if action == 1: #comprando açao
      trader.inventory.append(data[t])
      print('IA Trader bought: ', stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: #vendendo açao
      buy_price = trader.inventory.pop(0)

      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("IA Trader sold: ", stocks_price_format(data[t]), "Profit: " + stocks_price_format(data[t] - buy_price))

    if t == data_samples - 1:
      done = True
    else:
      done = False

    trader.memory.append((state, action, reward, next_state, done))

    state = next_state

    if done:
      print("###############")
      print("Total profit: {}".format(total_profit))
      print("###############")

    # if len(trader.memory) > batch_size:
    #   trader.batch_train(batch_size)

  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

Episode: 1/10


 11%|█▏        | 314/2767 [00:00<00:00, 3133.42it/s]

IA Trader bought:  $ 18.582144
IA Trader sold:  $ 17.354286 Profit: - $ 1.227858
IA Trader bought:  $ 17.952856
IA Trader bought:  $ 18.357500
IA Trader bought:  $ 16.089287
IA Trader bought:  $ 16.065357
IA Trader sold:  $ 16.366785 Profit: - $ 1.586071
IA Trader bought:  $ 16.315357
IA Trader sold:  $ 16.200714 Profit: - $ 2.156786
IA Trader sold:  $ 16.351429 Profit: $ 0.262142
IA Trader sold:  $ 16.333929 Profit: $ 0.268572
IA Trader sold:  $ 16.722143 Profit: $ 0.406786
IA Trader bought:  $ 16.678928
IA Trader bought:  $ 16.663929
IA Trader bought:  $ 16.434286
IA Trader bought:  $ 16.428213
IA Trader sold:  $ 15.930714 Profit: - $ 0.748215
IA Trader bought:  $ 16.100357
IA Trader bought:  $ 15.814286
IA Trader sold:  $ 15.877500 Profit: - $ 0.786429
IA Trader sold:  $ 15.373929 Profit: - $ 1.060357
IA Trader bought:  $ 15.001786
IA Trader bought:  $ 15.377857
IA Trader sold:  $ 15.418571 Profit: - $ 1.009642
IA Trader bought:  $ 15.301071
IA Trader sold:  $ 15.298214 Profit: - $ 

 23%|██▎       | 628/2767 [00:00<00:00, 2613.59it/s]

IA Trader sold:  $ 32.634998 Profit: $ 4.534998
IA Trader bought:  $ 32.529999
IA Trader bought:  $ 32.162498
IA Trader sold:  $ 32.147499 Profit: $ 4.862499
IA Trader bought:  $ 31.792500
IA Trader sold:  $ 31.825001 Profit: $ 2.997501
IA Trader sold:  $ 31.902500 Profit: $ 2.177500
IA Trader sold:  $ 32.027500 Profit: $ 2.737499
IA Trader bought:  $ 31.875000
IA Trader sold:  $ 31.687500 Profit: $ 0.072500
IA Trader sold:  $ 31.357500 Profit: - $ 0.412500
IA Trader sold:  $ 31.650000 Profit: - $ 0.307501
IA Trader bought:  $ 31.500000
IA Trader bought:  $ 31.422501
IA Trader sold:  $ 30.642500 Profit: - $ 2.607500
IA Trader sold:  $ 30.820000 Profit: - $ 1.295002
IA Trader sold:  $ 31.402500 Profit: - $ 0.869999
IA Trader bought:  $ 31.705000
IA Trader sold:  $ 32.127499 Profit: $ 1.567499
IA Trader sold:  $ 33.017502 Profit: $ 2.120003
IA Trader bought:  $ 32.687500
IA Trader bought:  $ 31.305000
IA Trader sold:  $ 31.290001 Profit: $ 0.052502
IA Trader bought:  $ 31.125000
IA Trade

...

 59%|█████▉    | 1628/2767 [00:01<00:00, 1196.50it/s]

 $ 55.775002
IA Trader bought:  $ 55.325001
IA Trader bought:  $ 54.582500
IA Trader bought:  $ 55.962502
IA Trader sold:  $ 56.602501 Profit: $ 10.622501
IA Trader sold:  $ 55.959999 Profit: $ 9.610001
IA Trader sold:  $ 54.470001 Profit: $ 7.477501
IA Trader bought:  $ 54.560001
IA Trader bought:  $ 54.592499
IA Trader bought:  $ 55.007500
IA Trader sold:  $ 54.415001 Profit: $ 6.827499
IA Trader bought:  $ 55.547501
IA Trader sold:  $ 55.105000 Profit: $ 7.272499
IA Trader sold:  $ 56.435001 Profit: $ 8.835003
IA Trader bought:  $ 56.814999
IA Trader sold:  $ 57.320000 Profit: $ 5.052498
IA Trader sold:  $ 58.017502 Profit: $ 6.240002
IA Trader sold:  $ 56.997501 Profit: $ 4.777500
IA Trader bought:  $ 55.942501
IA Trader sold:  $ 56.717499 Profit: $ 2.322498
IA Trader sold:  $ 54.090000 Profit: $ 0.330002
IA Trader sold:  $ 53.612499 Profit: - $ 0.260002
IA Trader bought:  $ 55.527500
IA Trader bought:  $ 54.340000
IA Trader bought:  $ 55.537498
IA Trader sold:  $ 55.297501 Profit:

 92%|█████████▏| 2552/2767 [00:01<00:00, 2374.17it/s]

IA Trader sold:  $ 115.010002 Profit: $ 23.810005
IA Trader sold:  $ 114.907501 Profit: $ 21.445000
IA Trader bought:  $ 114.607498
IA Trader sold:  $ 115.562500 Profit: $ 20.220001
IA Trader bought:  $ 118.275002
IA Trader sold:  $ 124.370003 Profit: $ 27.312500
IA Trader bought:  $ 124.824997
IA Trader sold:  $ 126.522499 Profit: $ 29.522499
IA Trader sold:  $ 125.010002 Profit: $ 30.200005
IA Trader sold:  $ 124.807503 Profit: $ 28.617500
IA Trader bought:  $ 129.039993
IA Trader sold:  $ 131.399994 Profit: $ 17.497498
IA Trader bought:  $ 120.959999
IA Trader sold:  $ 117.320000 Profit: $ 4.592499
IA Trader sold:  $ 113.489998 Profit: $ 4.114998
IA Trader sold:  $ 112.000000 Profit: - $ 1.010002
IA Trader sold:  $ 115.360001 Profit: $ 0.752502
IA Trader bought:  $ 115.540001
IA Trader bought:  $ 112.129997
IA Trader bought:  $ 110.339996
IA Trader sold:  $ 106.839996 Profit: - $ 11.435005
IA Trader sold:  $ 110.080002 Profit: - $ 14.744995
IA Trader bought:  $ 111.809998
IA Trader 

100%|██████████| 2767/2767 [00:01<00:00, 1811.20it/s]
